# Create Boundary Conditions of the 79NG Fjord GETM Setup

This notebook creates the GETM input files describing the boundary conditions of the 79NG fjord setup.
The 3D boundary conditions are given by the initial temperatures and salinities at the boundary points.

Notebook by Markus Reinert (IOW, 2023–2024, https://orcid.org/0000-0002-3761-8029).

In [ ]:
from datetime import datetime

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean

from tools.configuration import Configuration

In [ ]:
config = Configuration()

## Load the topography

In [ ]:
filename = config.get_file_path("getm/domain/bathymetry")
print(f"Loading topography from {filename!r}.")
topo = xr.open_dataset(filename)
topo

## Show the land/ocean mask

In [ ]:
fig, ax = plt.subplots(figsize=(7, 6), dpi=300)
ax.pcolormesh(topo.mask)
ax.set_title("GETM index 1 is between grid lines 0 and 1\nGETM index 11 is above/to the right of grid line 10")
ax.set_aspect("equal")
xticks = np.arange(0, topo.lon.size + 1, 10)
ax.set_xticks(xticks, np.where(xticks % 20 == 0, xticks, ""))
ax.set_yticks(np.arange(0, topo.lat.size + 1, 10))
ax.set_xticks(np.arange(topo.lon.size + 1), minor=True)
ax.set_yticks(np.arange(topo.lat.size + 1), minor=True)
ax.grid(which="minor", linewidth=0.5)
ax.grid(color="red", linewidth=0.5)

## Create the boundary info file

The file format is explained at https://getm.eu/bdys/articles/bdys.html.
Here's a compass for orientation:
|     |     |     |
| :-: | :-: | :-: |
| NW  |**N**| NE  |
|**W**|  ↑  |**E**|
| SW  |**S**| SE  |

### Get start and end indices of the open boundaries

In [ ]:
# North
# From the first ocean point (mask is True) to one before the last grid point,
# because the last grid point (NE corner) belongs to the eastern boundary
i_bdy_start_N = np.where(topo.mask.isel(lat=-1))[0][0]
i_bdy_end_N = topo.lon.size - 2

# East
# Full extent of the model grid
i_bdy_start_E = 0
i_bdy_end_E = topo.lat.size - 1

# South
# Analog to the northern boundary
i_bdy_start_S = np.where(topo.mask.isel(lat=0))[0][0]
i_bdy_end_S = topo.lon.size - 2

In [ ]:
n_bdy_N = i_bdy_end_N - i_bdy_start_N + 1
n_bdy_E = i_bdy_end_E - i_bdy_start_E + 1
n_bdy_S = i_bdy_end_S - i_bdy_start_S + 1
n_bdy_points = n_bdy_N + n_bdy_E + n_bdy_S
print("Open boundary consists of", n_bdy_points, "grid points.")

### Create the file content

Add 1 to every index, because GETM starts counting indices at 1 and not 0.

In [ ]:
boundary_info = f"""\
# no western boundary
0
# northern boundary
1
{topo.lat.size} {i_bdy_start_N + 1} {i_bdy_end_N + 1} 4 0
# eastern boundary over the full model domain
1
{topo.lon.size} {i_bdy_start_E + 1} {i_bdy_end_E + 1} 4 0
# southern boundary
1
1 {i_bdy_start_S + 1} {i_bdy_end_S + 1} 4 0
"""
print(boundary_info.strip())

### Save the file

In [ ]:
filename = config.get_file_path("getm/domain/bdyinfofile")
with open(filename, "w") as f:
    f.write(boundary_info)
print(f"Saved the boundary info as {filename!r}.")

## Create 2D boudary conditions

### Create the dataset

In [ ]:
time_string_0 = config.get_text("getm/time/start")
time_string_1 = config.get_text("getm/time/stop")
datetime_0 = datetime.strptime(time_string_0, "%Y-%m-%d %H:%M:%S")
datetime_1 = datetime.strptime(time_string_1, "%Y-%m-%d %H:%M:%S")
print(f"Model runs from {datetime_0} to {datetime_1}.")

In [ ]:
bdy_2D = xr.Dataset(
    {
        "elev": (
            ["time", "nbdy"],
            np.zeros((2, n_bdy_points)),
            {"long_name": "sea surface elevation", "units": "m"},
        ),
        "u": (
            ["time", "nbdy"],
            np.zeros((2, n_bdy_points)),
            {"long_name": "zonal velocity", "units": "m/s"},
        ),
        "v": (
            ["time", "nbdy"],
            np.zeros((2, n_bdy_points)),
            {"long_name": "meridional velocity", "units": "m/s"},
        ),
    },
    coords={
        "nbdy": (["nbdy"], np.arange(n_bdy_points)),
        "time": (["time"], [datetime_0, datetime_1]),
    },
    attrs={
        "title": "Boundary conditions (2D) for the 79NG fjord GETM setup",
        "author": "Markus Reinert (ORCID: 0000-0002-3761-8029)",
        "institution": "Leibniz Institute for Baltic Sea Research Warnemuende (IOW), Germany",
        "description": "Boundary conditions are given by constant zero elevation and velocity.",
    },
)
bdy_2D

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/m2d/bdyfile_2d")
bdy_2D.to_netcdf(
    filename,
    unlimited_dims=["time"],
    encoding={
        "u": {"_FillValue": None},
        "v": {"_FillValue": None},
        "elev": {"_FillValue": None},
        "time": {"units": "seconds since 2000-01-01"},
    },
)
print(f"Saved the 2D boundary conditions as {filename!r}.")

## Create 3D boundary conditions

### Load the corresponding inital conditions

In [ ]:
filename = config.get_file_path("getm/temp/temp_file")
assert filename == config.get_file_path("getm/salt/salt_file"), "filenames of temperature and salinity initial conditions differ"
print(f"Loading initial conditions from {filename!r}.")
init = xr.open_dataset(filename).squeeze()
variables = [var for var in init.variables if init[var].dims == ("zax", "lat", "lon")]
print("Contains", *variables)
init

### Create the 3D boundary dataset

In [ ]:
bdy_3D = xr.Dataset(
    {
        var: (
            ["time", "nbdy", "zax"],
            np.full((2, n_bdy_points, init.zax.size), np.nan),
            init[var].attrs,
        ) for var in variables
    },
    coords={
        "nbdy": (["nbdy"], bdy_2D.nbdy.data),
        "time": (["time"], bdy_2D.time.data),
        "zax": (["zax"], -init.zax.data, {"long_name": "z-axis", "units": init.zax.units, "positive": "up"}),
    },
    attrs={
        "title": "Boundary conditions (3D) for the 79NG fjord GETM setup",
        "author": bdy_2D.author,
        "institution": bdy_2D.institution,
        "description": "Boundary conditions are equal to the inital conditions for S and T.",
        "source": init.source,
    },
)
bdy_3D

### Fill the dataset

In [ ]:
for var in variables:
    bdy_N = bdy_3D[var].isel(nbdy=slice(n_bdy_N))
    bdy_N[...] = init[var].isel(lat=-1, lon=slice(i_bdy_start_N, i_bdy_end_N + 1)).data.T
    bdy_E = bdy_3D[var].isel(nbdy=slice(n_bdy_N, n_bdy_N + n_bdy_E))
    bdy_E[...] = init[var].isel(lon=-1).data.T
    bdy_S = bdy_3D[var].isel(nbdy=slice(n_bdy_N + n_bdy_E, n_bdy_N + n_bdy_E + n_bdy_S))
    bdy_S[...] = init[var].isel(lat=0, lon=slice(i_bdy_start_S, i_bdy_end_S + 1)).data.T
    assert all(bdy_3D.nbdy == np.concatenate([bdy_N.nbdy, bdy_E.nbdy, bdy_S.nbdy])), "not all boundary points covered"

### Show the dataset

In [ ]:
# Extract the bathymetry and lat/lon-coordinates of the boundaries
z_seabed = -np.concatenate([
    topo.bathymetry.isel(lat=0, lon=slice(i_bdy_start_S, i_bdy_end_S + 1)),
    topo.bathymetry.isel(lon=-1),
    topo.bathymetry.isel(lat=-1, lon=slice(i_bdy_end_N, i_bdy_start_N - 1, -1)),
])
l_bdy = np.concatenate([
    topo.lon[i_bdy_start_S : i_bdy_end_S + 1],
    topo.lat,
    topo.lon[i_bdy_end_N : i_bdy_start_N - 1 : -1],
])

In [ ]:
fig, axs = plt.subplots(2, sharex=True, sharey=True, constrained_layout=True, figsize=(8, 5), dpi=300)
fig.suptitle("3D Boundary Conditions", weight="bold")

for ax, var in zip(axs, variables):
    data = bdy_3D[var].data[0]
    # Re-assemble the boundary data in counter-clockwise direction from South to North
    data = np.concatenate((data[n_bdy_N + n_bdy_E:], data[n_bdy_N : n_bdy_N + n_bdy_E], data[n_bdy_N - 1 : : -1]))
    im = ax.pcolormesh(
        bdy_3D.nbdy, bdy_3D.zax, np.where(bdy_3D.zax.data[:, np.newaxis] >= z_seabed, data.T, np.nan),
        shading="nearest",
        cmap=cmocean.cm.thermal if var == "temp" else cmocean.cm.haline if var == "salt" else None,
    )
    fig.colorbar(im, ax=ax)
    ax.plot(bdy_3D.nbdy, z_seabed, "tab:grey", lw=1)
    # Mark the transitions between sections
    kwargs = dict(color="r", lw=1, ls="--")
    ax.axvline(n_bdy_S - 0.5, **kwargs)
    ax.axvline(n_bdy_S + n_bdy_E - 0.5, **kwargs)
    ax.set_ylim(z_seabed.min(), 0)
    ax.set_title(f"{bdy_3D[var].long_name.capitalize()} [{bdy_3D[var].units}]")
    ax.set_ylabel(f"{bdy_3D.zax.long_name} [{bdy_3D.zax.units}]")
ax.set_xlabel("Boundary points from South over East to North (counter-clockwise)")

# Put x-labels at each degree longitude and at every fifth degree latitude
indices = (
    [np.argmin(abs(l_bdy[:n_bdy_S] + lon)) for lon in range(19, 14, -1)] +
    [np.argmin(abs(l_bdy - lat)) for lat in np.arange(79.2, 80.3, 0.2)] + 
    [np.argmin(abs(l_bdy[n_bdy_S + n_bdy_E:] + lon)) + n_bdy_S + n_bdy_E for lon in (15, 16)]
)
labels = [f"{-l:.0f}°W" if l < 0 else f"\n{l:.1f}°N" for l in l_bdy[indices]]
ax.set_xticks(indices, labels)

fig.savefig("figures/bdy_3d.png")

### Save the dataset

In [ ]:
filename = config.get_file_path("getm/m3d/bdyfile_3d")
bdy_3D.to_netcdf(
    filename,
    unlimited_dims=["time"],
    encoding={
        "zax": {"_FillValue": None},
        **{var: {"_FillValue": None} for var in variables},
        "time": {"units": "seconds since 2000-01-01"},
    },
)
print(f"Saved the 3D boundary conditions as {filename!r}.")